PROVIDED WITH MEMO OF THE FULLY CLEAN DATASET
WE WILL READ THE DATASET AND THEN GO ON TO FULLY DESCRIBING THE DATA

In [3]:
#load cleansed dataset to read
import pandas as pd
df = pd.read_excel('_Delta Airlines Loyalty Customer Cleansed Data.xlsx')
df.head()

,Record Creation Date,Join Date,Frequent Flier Number,First Name,Last Name,Inquiry Type,Most Recent Flight #,Lounge Used?,Departing Airport,Arrival Airport,Planned Snack?,Additional Snack?,# of Included Checked Bags,Total # of Checked Bags,Flight Delayed?,Lead Source,Online Check-In,Add-On Spend
0,2024-02-27,1988-07-08,7234617746,Debbie,Spears,Flight Status,G35206241,False,HND,HND,False,True,2,7,True,Travel Agency,False,21.14
1,2021-03-07,1988-07-08,7234617746,Debbie,Spears,New Flight,V81311927,True,JFK,SFO,False,True,1,6,False,Travel Agency,False,95.57
2,2023-01-31,1988-07-08,7234617746,Debbie,Spears,Cancel Flight,H82295055,False,CDG,ORD,False,False,2,5,True,Discounted Promo,False,8.66
3,2021-08-21,2022-06-16,551121699,Kyle,Boone,New Flight,E39886834,True,SEA,DEN,False,True,3,8,False,Discounted Promo,False,82.59
4,2022-03-01,2022-06-16,551121699,Kyle,Boone,New Flight,R04390271,True,SFO,DXB,False,False,3,5,False,Discounted Promo,False,26.70


In [5]:
# Analyze frequent flyer behavior
frequent_flyer_inquiry = df.groupby('Frequent Flier Number')['Inquiry Type'].value_counts().unstack().fillna(0)

# Display top frequent flyers and the types of inquiries they made
frequent_flyer_inquiry.head()


Inquiry Type,Cancel Flight,Flight Deal,Flight Status,New Flight
Frequent Flier Number,,,,
5445899,0.0,2.0,0.0,1.0
5911884,2.0,4.0,3.0,1.0
8157372,3.0,0.0,0.0,0.0
10806849,0.0,1.0,1.0,1.0
12109458,2.0,1.0,0.0,0.0


In [7]:
# Calculate the average and total Add-On Spend by inquiry type
add_on_spend = df.groupby('Inquiry Type')['Add-On Spend'].agg(['mean', 'sum', 'count'])

# Display the results
add_on_spend


,mean,sum,count
Inquiry Type,,,
Cancel Flight,104.388472,231638.02,2219
Flight Deal,106.763279,240964.72,2257
Flight Status,106.957241,250066.03,2338
New Flight,105.226583,252228.12,2397


In [9]:
# Calculate the percentage of delayed flights by airport
delayed_flights_by_airport = df.groupby('Departing Airport')['Flight Delayed?'].mean()

# Display the top airports with the most delays
delayed_flights_by_airport.sort_values(ascending=False).head()


Departing Airport
SYD    0.547264
MIA    0.526611
CDG    0.522449
HND    0.520492
DEN    0.519774
Name: Flight Delayed?, dtype: float64

In [11]:
# Calculate the percentage of customers using lounges
lounge_usage = df['Lounge Used?'].value_counts(normalize=True)

# Display the results
lounge_usage


Lounge Used?
True     0.503311
False    0.496689
Name: proportion, dtype: float64

In [13]:
# Count the number of flights per route (from departing to arrival airport)
flight_routes = df.groupby(['Departing Airport', 'Arrival Airport']).size().reset_index(name='Flight Count')

# Display top flight routes
flight_routes.sort_values(by='Flight Count', ascending=False).head()


,Departing Airport,Arrival Airport,Flight Count
131,LAX,JFK,130
116,JFK,LAX,112
134,LAX,MIA,99
118,JFK,MIA,97
194,SEA,LAX,96


In [15]:
# Convert the date columns to datetime if they aren't already
df['Record Creation Date'] = pd.to_datetime(df['Record Creation Date'])

# Group by month and analyze add-on spend
monthly_spend = df.groupby(df['Record Creation Date'].dt.to_period('M'))['Add-On Spend'].sum()

# Display the results
monthly_spend


Record Creation Date
2021-01    24449.98
2021-02    18043.06
2021-03    22919.54
2021-04    19655.62
2021-05    17117.11
2021-06    22799.66
2021-07    17771.42
2021-08    26812.61
2021-09    20214.97
2021-10    22911.09
2021-11    21598.91
2021-12    24015.91
2022-01    21439.97
2022-02    22282.32
2022-03    21285.47
2022-04    20392.25
2022-05    20321.64
2022-06    21866.41
2022-07    20629.48
2022-08    21118.83
2022-09    22346.81
2022-10    23559.72
2022-11    19431.37
2022-12    25036.47
2023-01    20348.37
2023-02    15121.50
2023-03    22062.81
2023-04    19749.88
2023-05    20959.75
2023-06    21494.88
2023-07    22439.37
2023-08    24149.24
2023-09    21321.24
2023-10    19736.52
2023-11    22484.65
2023-12    24926.85
2024-01    21706.44
2024-02    22738.06
2024-03    16883.63
2024-04    17448.75
2024-05    23649.44
2024-06    20767.52
2024-07    22474.18
2024-08    21224.68
2024-09    24336.16
2024-10    10852.35
Freq: M, Name: Add-On Spend, dtype: float64

In [19]:
# Summarize findings in each area based on previous analysis
findings_summary = {
    'Customer Behavior': {
        'Frequent Flyer Inquiries': frequent_flyer_inquiry.head(),  # Top inquiries by frequent flyers
        'Insight': "High-value customers frequently inquire about flight statuses, indicating opportunities to enhance proactive communication for frequent travelers."
    },
    'Add-On Spending': {
        'Add-On Spend by Inquiry Type': add_on_spend,  # Spending per inquiry type
        'Insight': "Certain inquiry types, such as new flight requests, are linked to higher add-on spends, highlighting potential upsell opportunities."
    },
    'Flight Delays': {
        'Delays by Airport': delayed_flights_by_airport.sort_values(ascending=False).head(),  # Top airports by delay frequency
        'Insight': "Specific airports show a higher rate of delays, which could indicate logistical challenges at these locations."
    },
    'Lounge Usage': {
        'Lounge Usage Rate': lounge_usage,  # Percentage of customers using lounges
        'Insight': "A significant portion of frequent flyers use lounges, suggesting that lounge services are a valuable part of the loyalty program."
    },
    'Flight Routes': {
        'Popular Flight Routes': flight_routes.sort_values(by='Flight Count', ascending=False).head(),  # Most frequent routes
        'Insight': "Certain routes are highly frequented, indicating opportunities for optimizing scheduling and capacity on these routes."
    },
    'Monthly Spend': {
        'Monthly Add-On Spend': monthly_spend,  # Add-on spending trends over time
        'Insight': "There are seasonal fluctuations in add-on spending, suggesting that targeted promotions could increase revenue during low-spending months."
    }
}

# Display summary for verification
for key, value in findings_summary.items():
    print(f"{key}:\n")
    for sub_key, sub_value in value.items():
        print(f"{sub_key}: {sub_value}\n")
    print("\n" + "-"*50 + "\n")


Customer Behavior:

Frequent Flyer Inquiries: Inquiry Type           Cancel Flight  Flight Deal  Flight Status  New Flight
Frequent Flier Number                                                       
5445899                          0.0          2.0            0.0         1.0
5911884                          2.0          4.0            3.0         1.0
8157372                          3.0          0.0            0.0         0.0
10806849                         0.0          1.0            1.0         1.0
12109458                         2.0          1.0            0.0         0.0

Insight: High-value customers frequently inquire about flight statuses, indicating opportunities to enhance proactive communication for frequent travelers.


--------------------------------------------------

Add-On Spending:

Add-On Spend by Inquiry Type:                      mean        sum  count
Inquiry Type                               
Cancel Flight  104.388472  231638.02   2219
Flight Deal    106.76327

In [25]:
# Exporting the summarized insights to a text file for documentation
with open('delta_findings_summary.txt', 'w') as file:
    for key, value in findings_summary.items():
        file.write(f"{key}:\n")
        for sub_key, sub_value in value.items():
            file.write(f"{sub_key}: {sub_value}\n")
        file.write("\n" + "-"*50 + "\n")
